# XGBoost

Editor: Lu Lifei

Test part: please check, thanks

Evaluation part: hope one of our team members could check

Model part: based on @Julian '2018_05_09_b_DM2_DMC_XGBoost.ipynb'

## Model
### 1. Import and load

In [25]:
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

In [61]:
# Import cluster identifier

sales = pd.read_csv('/Users/effylu/Downloads/data_v0.1_sales.csv')
big_key = sales['key'][sales['cluster'] == "big"]

In [62]:
# Import datasets

X_full = pickle.load(open('/Users/effylu/Downloads/XY_flat/X_flat.pkl', 'rb'))
Y_full = pickle.load(open('/Users/effylu/Downloads/XY_flat/Y_flat.pkl', 'rb'))

In [63]:
#drop_cols = X_full.columns[12+14:12+14+14]
#X_full = X_full.drop(drop_cols, axis=1)
#print(X_full.columns)

### 2. Prepare

In [64]:
# Find rows which belong to cluster 'big' in X_full

X_full['key'] = X_full['key'].astype(str)
X_big = X_full[X_full['key'].isin(big_key.astype(str))]
X_big = X_big.reset_index(drop=True)

In [65]:
# Find rows which belong to cluster 'big' in Y_full

Y_full['key'] = Y_full['key'].astype(str)
Y_big = Y_full[Y_full['key'].isin(big_key.astype(str))]
Y_big = Y_big.reset_index(drop=True)

In [66]:
# Split the X_big and Y_big into traing and test

X_big['month'] = pd.DatetimeIndex(X_big['date']).month
Y_big['month'] = pd.DatetimeIndex(Y_big['date']).month
X_big_train = X_big.loc[X_big['month'] != 1]
Y_train = Y_big.loc[Y_big['month'] != 1]['sales']
X_big_test = X_big.loc[X_big['month'] == 1]
Y_test = Y_big.loc[Y_big['month'] == 1]['sales']

In [67]:
# Prepare the data for fitting the input of the model

drop_x_cols = ['key', 'pid_x', 'size_x', 'color', 'brand', 'rrp', 'date', 'day_of_week', 
               'mainCategory', 'category', 'subCategory', 'releaseDate', 
               'rrp', 'price', 'month']
X_train = X_big_train.drop(drop_x_cols, axis=1)
X_test = X_big_test.drop(drop_x_cols, axis=1)
print(X_train.columns)
X_train = X_train.as_matrix()
X_train = np.delete(X_train, np.s_[14:28], axis=1)
Y_train = Y_train.as_matrix()
X_test = X_test.as_matrix()
X_test = np.delete(X_test, np.s_[14:28], axis=1)
Y_test = Y_test.as_matrix()

Index(['last_1_day_sales', 'last_2_day_sales', 'last_3_day_sales',
       'last_4_day_sales', 'last_5_day_sales', 'last_6_day_sales',
       'last_7_day_sales', 'last_8_day_sales', 'last_9_day_sales',
       'last_10_day_sales', 'last_11_day_sales', 'last_12_day_sales',
       'last_13_day_sales', 'last_14_day_sales', 'last_15_day_sales',
       'last_16_day_sales', 'last_17_day_sales', 'last_18_day_sales',
       'last_19_day_sales', 'last_20_day_sales', 'last_21_day_sales',
       'last_22_day_sales', 'last_23_day_sales', 'last_24_day_sales',
       'last_25_day_sales', 'last_26_day_sales', 'last_27_day_sales',
       'last_28_day_sales', 'is_eleventh', 'is_crazy_day', 'day_Friday',
       'day_Monday', 'day_Saturday', 'day_Sunday', 'day_Thursday',
       'day_Tuesday', 'day_Wednesday', 'days_since_release', 'price_diff',
       'color_beige', 'color_blau', 'color_braun', 'color_gelb', 'color_gold',
       'color_grau', 'color_gruen', 'color_khaki', 'color_lila',
       'color_orange

In [68]:
X_test.shape

(90117, 81)

### 3. Train model

In [69]:
# Training the model

model = XGBRegressor()
model.fit(X_train, Y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

## Test

In [70]:
# Test the model
# Only the sale unit of the first day for each item is right int the 'X_test'
# Select the row 'on Jan 1st'

X_Jan1 = X_test[0:1,:]
for i in range(int(len(X_test)/31-1)):
    X_Jan1 = np.vstack([X_Jan1, X_test[(31+i*31):(32+i*31),:]])
print(X_Jan1)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  2. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 3.  0.  0. ...,  0.  0.  0.]]


In [71]:
print(X_Jan1.shape)

(2907, 81)


In [72]:
# Predict the sales unit 'on Jan 1st' for each items
# Change the format of the prediction results on Jan_1st

Y_Jan1 = model.predict(X_Jan1)
prediction_1 = np.asarray([round(value) for value in Y_Jan1])
prediction_1 = np.reshape(prediction_1, (len(prediction_1),1))
print(prediction_1)

[[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 0.]
 [ 0.]
 [ 1.]]


In [73]:
# Delete the 'last_28_day_sales'
# Add the prediction results as the 'last_1_day_sales'

X_Jan = X_Jan1
X_Jan = np.delete(X_Jan, np.s_[13:14], axis=1)
X_Jan = np.append(prediction_1, X_Jan, axis=1)
print(X_Jan)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 1.  3.  0. ...,  0.  0.  0.]]


In [74]:
# Add the process above into a loop
# Predict the sales units eery day in January for each item

predictions = prediction_1
for i in range(30):
    Y_Jan = model.predict(X_Jan)
    prediction = np.asarray([round(value) for value in Y_Jan])
    prediction = np.reshape(prediction, (len(prediction),1))
    predictions = np.append(predictions, prediction, axis=1)
    X_Jan = np.delete(X_Jan, np.s_[13:14], axis=1)
    X_Jan = np.append(prediction, X_Jan, axis=1)
print(predictions)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 1.  1.  1. ...,  0.  0.  0.]]


## Evaluation

### 1. Restructure the predictions

In [75]:
# Reshape predictions: row ->'big' items, columns -> date

column_name = X_big_test['date'].unique().astype(str)
row_name = X_big_test['key'].unique().astype(str)

In [76]:
# Aggregate sales for each day each item.

pred_agg = predictions
agg_sum = predictions[:,0]
for i in range(len(column_name)-1):
    agg_sum = pred_agg[:,i] + predictions[:, i+1]
    pred_agg[:, i+1] = agg_sum
print(pred_agg)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 1.  2.  3. ...,  3.  3.  3.]]


### 2. Test_0.csv
#### 2.1. Prepare 

In [77]:
# Load the test data

test_0 = pd.read_csv('/Users/effylu/Downloads/test_0.csv')

In [78]:
# Add 'key' for test data by merging 'pid' and 'size'
# Select useful attributes

test_0["key"] = test_0["pid"].map(int).map(str) + test_0["size"]
test_0_big = test_0.loc[test_0['key'].isin(row_name)]
subtest_0_big = test_0_big[['key','stock','sold_out_date']]
test = np.asarray(subtest_0_big)
print(test)

[['10063L' 1.0 '2018-01-12']
 ['10063M' 8.0 '2018-01-26']
 ['10063XL' 1.0 '2018-01-03']
 ..., 
 ['2286944' 2.0 '2018-01-31']
 ['2286945' 2.0 '2018-01-25']
 ['22872M ( 140-152 )' 4.0 '2018-01-25']]


#### 2.2. Match the 'test' with the 'pred_agg'

In [79]:
# Define arrays for storing the predicted day and date

pred_day = np.zeros((len(test),1), dtype=int)
pred_date = np.asarray(test_0_big[['sold_out_date']])

for i in range(len(test)):
    
    # 'key' is the key for each test item
    key = test[i,0]
    # Find the index of the item in predictions sharing the same key
    index = 0
    
    # Match the items in 'test' with the items in 'pred_agg'
    # Retern the index of the item in the 'pred_agg'
    for j in range(len(row_name)):
        if row_name[j] == key:
            index = j
            break

    # Match
    if test[i,1] < pred_agg[index,0]:
        pred_day[i,0] = 1
        pred_date[i,0] = column_name[0]
        continue
    if test[i,1] > pred_agg[index,30]:
        pred_day[i,0] = 15
        pred_date[i,0] = column_name[14]
        continue
    for k in range(len(pred_agg[0])):
        if pred_agg[index,k] - test[i,1] >=0:
            pred_day[i,0] = k+1
            pred_date[i,0] = column_name[k]
            break

#### 2.3. Error Calculation

In [80]:
# Find the day of the real sold out date

day = [i[-2:] for i in test[:,2].tolist()]
real_day = np.reshape(np.asarray(list(map(int,day))), (len(test),1))
print(real_day)

[[12]
 [26]
 [ 3]
 ..., 
 [31]
 [25]
 [25]]


In [81]:
# Show the predicted sold out date

print(pred_day)

[[15]
 [15]
 [15]
 ..., 
 [15]
 [15]
 [15]]


In [82]:
# Error calculation

error = sqrt(np.sum(np.abs(np.subtract(pred_day, real_day))))
print(error)

142.57980221616245


#### 2.4. Visualize result

In [83]:
# Visualize the result in dataframe

result = np.append(test, pred_date, axis=1)
result_column =['key','stock','real_sold_out_day','predicted_sold_out_day']
prediction_result = pd.DataFrame(result, columns=result_column)
prediction_result

,key,stock,real_sold_out_day,predicted_sold_out_day
0,10063L,1,2018-01-12,2018-01-15
1,10063M,8,2018-01-26,2018-01-15
2,10063XL,1,2018-01-03,2018-01-15
3,100652 ( 37-39 ),20,2018-01-24,2018-01-15
4,100654 ( 43-45 ),3,2018-01-18,2018-01-15
5,10073L ( 40/42 ),9,2018-01-04,2018-01-15
6,10073XL ( 44/46 ),1,2018-01-01,2018-01-15
7,10084M,3,2018-01-30,2018-01-15
8,1011944 2/3,2,2018-01-26,2018-01-15
9,1011945 1/3,3,2018-01-31,2018-01-15


### 3. Test_1.csv

In [84]:
# Load the test data
test_0 = pd.read_csv('/Users/effylu/Downloads/test_1.csv')

# Add 'key' for test data by merging 'pid' and 'size'
# Select useful attributes
test_0["key"] = test_0["pid"].map(int).map(str) + test_0["size"]
test_0_big = test_0.loc[test_0['key'].isin(row_name)]
subtest_0_big = test_0_big[['key','stock','sold_out_date']]
test = np.asarray(subtest_0_big)

In [85]:
# Define arrays for storing the predicted day and date

pred_day = np.zeros((len(test),1), dtype=int)
pred_date = np.asarray(test_0_big[['sold_out_date']])

for i in range(len(test)):
    
    # 'key' is the key for each test item
    key = test[i,0]
    # Find the index of the item in predictions sharing the same key
    index = 0
    
    # Match the items in 'test' with the items in 'pred_agg'
    # Retern the index of the item in the 'pred_agg'
    for j in range(len(row_name)):
        if row_name[j] == key:
            index = j
            break
    
    # Match
    if test[i,1] < pred_agg[index,0]:
        pred_day[i,0] = 1
        pred_date[i,0] = column_name[0]
        continue
    if test[i,1] > pred_agg[index,30]:
        pred_day[i,0] = 15
        pred_date[i,0] = column_name[14]
        continue
    for k in range(len(pred_agg[0])):
        if pred_agg[index,k] - test[i,1] >=0:
            pred_day[i,0] = k+1
            pred_date[i,0] = column_name[k]
            break

In [86]:
# Error calculation

day = [i[-2:] for i in test[:,2].tolist()]
real_day = np.reshape(np.asarray(list(map(int,day))), (len(test),1))
error = sqrt(np.sum(np.abs(np.subtract(pred_day, real_day))))
print(error)

142.55525244620065


### 4. Test_2.csv

In [87]:
# Load the test data
test_0 = pd.read_csv('/Users/effylu/Downloads/test_2.csv')

# Add 'key' for test data by merging 'pid' and 'size'
# Select useful attributes
test_0["key"] = test_0["pid"].map(int).map(str) + test_0["size"]
test_0_big = test_0.loc[test_0['key'].isin(row_name)]
subtest_0_big = test_0_big[['key','stock','sold_out_date']]
test = np.asarray(subtest_0_big)

In [88]:
# Define arrays for storing the predicted day and date

pred_day = np.zeros((len(test),1), dtype=int)
pred_date = np.asarray(test_0_big[['sold_out_date']])

for i in range(len(test)):
    
    # 'key' is the key for each test item
    key = test[i,0]
    # Find the index of the item in predictions sharing the same key
    index = 0
    
    # Match the items in 'test' with the items in 'pred_agg'
    # Retern the index of the item in the 'pred_agg'
    for j in range(len(row_name)):
        if row_name[j] == key:
            index = j
            break
    
    # Match
    if test[i,1] < pred_agg[index,0]:
        pred_day[i,0] = 1
        pred_date[i,0] = column_name[0]
        continue
    if test[i,1] > pred_agg[index,30]:
        pred_day[i,0] = 15
        pred_date[i,0] = column_name[14]
        continue
    for k in range(len(pred_agg[0])):
        if pred_agg[index,k] - test[i,1] >=0:
            pred_day[i,0] = k+1
            pred_date[i,0] = column_name[k]
            break

In [89]:
# Error calculation

day = [i[-2:] for i in test[:,2].tolist()]
real_day = np.reshape(np.asarray(list(map(int,day))), (len(test),1))
error = sqrt(np.sum(np.abs(np.subtract(pred_day, real_day))))
print(error)

143.24803663575986


### 5. Test_3.csv

In [90]:
# Load the test data
test_0 = pd.read_csv('/Users/effylu/Downloads/test_3.csv')

# Add 'key' for test data by merging 'pid' and 'size'
# Select useful attributes
test_0["key"] = test_0["pid"].map(int).map(str) + test_0["size"]
test_0_big = test_0.loc[test_0['key'].isin(row_name)]
subtest_0_big = test_0_big[['key','stock','sold_out_date']]
test = np.asarray(subtest_0_big)

In [91]:
# Define arrays for storing the predicted day and date

pred_day = np.zeros((len(test),1), dtype=int)
pred_date = np.asarray(test_0_big[['sold_out_date']])

for i in range(len(test)):
    
    # 'key' is the key for each test item
    key = test[i,0]
    # Find the index of the item in predictions sharing the same key
    index = 0
    
    # Match the items in 'test' with the items in 'pred_agg'
    # Retern the index of the item in the 'pred_agg'
    for j in range(len(row_name)):
        if row_name[j] == key:
            index = j
            break
    
    # Match
    if test[i,1] < pred_agg[index,0]:
        pred_day[i,0] = 1
        pred_date[i,0] = column_name[0]
        continue
    if test[i,1] > pred_agg[index,30]:
        pred_day[i,0] = 15
        pred_date[i,0] = column_name[14]
        continue
    for k in range(len(pred_agg[0])):
        if pred_agg[index,k] - test[i,1] >=0:
            pred_day[i,0] = k+1
            pred_date[i,0] = column_name[k]
            break

In [92]:
# Error calculation

day = [i[-2:] for i in test[:,2].tolist()]
real_day = np.reshape(np.asarray(list(map(int,day))), (len(test),1))
error = sqrt(np.sum(np.abs(np.subtract(pred_day, real_day))))
print(error)

143.07340773183535


### 6. Test_4.csv

In [93]:
# Load the test data
test_0 = pd.read_csv('/Users/effylu/Downloads/test_4.csv')

# Add 'key' for test data by merging 'pid' and 'size'
# Select useful attributes
test_0["key"] = test_0["pid"].map(int).map(str) + test_0["size"]
test_0_big = test_0.loc[test_0['key'].isin(row_name)]
subtest_0_big = test_0_big[['key','stock','sold_out_date']]
test = np.asarray(subtest_0_big)

In [94]:
# Define arrays for storing the predicted day and date

pred_day = np.zeros((len(test),1), dtype=int)
pred_date = np.asarray(test_0_big[['sold_out_date']])

for i in range(len(test)):
    
    # 'key' is the key for each test item
    key = test[i,0]
    # Find the index of the item in predictions sharing the same key
    index = 0
    
    # Match the items in 'test' with the items in 'pred_agg'
    # Retern the index of the item in the 'pred_agg'
    for j in range(len(row_name)):
        if row_name[j] == key:
            index = j
            break
    
    # Match
    if test[i,1] < pred_agg[index,0]:
        pred_day[i,0] = 1
        pred_date[i,0] = column_name[0]
        continue
    if test[i,1] > pred_agg[index,30]:
        pred_day[i,0] = 15
        pred_date[i,0] = column_name[14]
        continue
    for k in range(len(pred_agg[0])):
        if pred_agg[index,k] - test[i,1] >=0:
            pred_day[i,0] = k+1
            pred_date[i,0] = column_name[k]
            break

In [95]:
# Error calculation

day = [i[-2:] for i in test[:,2].tolist()]
real_day = np.reshape(np.asarray(list(map(int,day))), (len(test),1))
error = sqrt(np.sum(np.abs(np.subtract(pred_day, real_day))))
print(error)

142.13725760686393
